## Importing the libraries 

In [1]:
import pandas as pd
import numpy as np

## Load the data

In [22]:
path_to_file = '/content/sample_data/workshop_orders.csv'  # write your path to workshop_orders.csv

df = pd.read_csv(path_to_file)# ...
df.shape

(28028, 4)

## EDA

In [3]:
df.head()

,customer_id,order_status,order_amount,order_created_at
0,9d1adcfb-288b-42a2-93c0-3e6ab8df2205,approved,0.99,2022-08-08 02:00:50
1,b1bbe434-6d4e-46e5-9e54-6e785e0ad047,declined,29.99,2022-09-13 12:55:49
2,30ede893-594e-4fc1-9c5c-e28689af78d8,declined,29.99,2022-07-30 16:22:02
3,d66d68e1-e1f5-4a79-961a-2349d5bfc444,approved,0.99,2022-07-28 18:47:20
4,d1b56a92-19c6-4759-ab10-4045d4c9e79a,approved,29.99,2022-08-23 12:00:48


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19315 entries, 0 to 28027
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       19315 non-null  object 
 1   order_status      19315 non-null  object 
 2   order_amount      19315 non-null  float64
 3   order_created_at  19315 non-null  object 
dtypes: float64(1), object(3)
memory usage: 754.5+ KB


In [7]:
df["order_amount"].value_counts()

0.99     10092
29.99     9223
Name: order_amount, dtype: int64

In [8]:
df["order_status"].value_counts()

approved    17713
refunded     1602
Name: order_status, dtype: int64

In [9]:
df.isna().sum()

customer_id         0
order_status        0
order_amount        0
order_created_at    0
dtype: int64

In [10]:
set(df.duplicated())

{False}

## Data transformation

Delete declined transactions.


In [23]:
df = df[df.order_status != "declined"]

In [5]:
df.head()

,customer_id,order_status,order_amount,order_created_at
0,9d1adcfb-288b-42a2-93c0-3e6ab8df2205,approved,0.99,2022-08-08 02:00:50
3,d66d68e1-e1f5-4a79-961a-2349d5bfc444,approved,0.99,2022-07-28 18:47:20
4,d1b56a92-19c6-4759-ab10-4045d4c9e79a,approved,29.99,2022-08-23 12:00:48
6,3e58c43f-85eb-4c70-9a21-11f8e3d6481d,approved,29.99,2022-07-29 17:05:51
8,5443350f-6291-4171-868b-df07cc316055,approved,0.99,2022-08-11 10:54:17


For convenience, we will transform our data into single-user level (not single order).

In [26]:
df = df.sort_values(by=["customer_id"])

In [34]:
df.loc[df['order_status'] == "refunded", 'order_amount'] = -1 * df.loc[df['order_status'] == "refunded", 'order_amount']

In [39]:
df.head(10)

,customer_id,order_status,order_amount,order_created_at
14676,000230b4-1c56-4959-92be-cdcca793c04e,approved,0.99,2022-08-18 13:39:37
13364,000230b4-1c56-4959-92be-cdcca793c04e,approved,29.99,2022-09-20 13:40:48
11299,000230b4-1c56-4959-92be-cdcca793c04e,approved,29.99,2022-08-21 13:39:55
21909,0006303d-0c89-43da-b18a-12d7b930260a,approved,0.99,2022-07-17 18:57:29
417,0006303d-0c89-43da-b18a-12d7b930260a,refunded,-29.99,2022-07-20 18:57:47
17102,0011e335-d60e-40dd-98a6-c67daeb955a8,approved,0.99,2022-08-23 19:41:26
17380,0011e335-d60e-40dd-98a6-c67daeb955a8,approved,29.99,2022-08-26 19:41:49
12089,001669a4-1baf-4b34-82c1-6e3340ec86e1,approved,0.99,2022-07-14 09:30:47
7502,0022d1a2-f448-4610-b01f-e817e116e741,approved,0.99,2022-07-23 12:12:06
14545,0022d1a2-f448-4610-b01f-e817e116e741,approved,29.99,2022-07-26 12:12:47


In [52]:
df2 = df.groupby('customer_id')["order_amount"].apply(list)

In [56]:
customer_dict = df2.to_dict()

## Calculating metrics

In [59]:
res_metrics = {
    "CR_from_trial": 0,
    "Refund_rate": 0,
    "Rebill_rate": 0,
    "Commisions": 0
}

CR from trial = number of converted to full charge users / total amount of users

In [96]:
num = 0
for cust in customer_dict:
  if len(customer_dict[cust]) > 1:
    num += 1

print(len(customer_dict), num)
res_metrics['CR_from_trial'] = num / len(customer_dict)# ...
res_metrics['CR_from_trial']

10092 6109


0.6053309552120492

Refund rate = amount of refunded / total earned

In [82]:
sum_refunded, sum_got = 0, 0
for cust in customer_dict:
  sum_got += sum([elm for elm in customer_dict[cust] if elm > 0])
  sum_refunded += sum([abs(elm) for elm in customer_dict[cust] if elm < 0])


res_metrics['Refund_rate'] = sum_refunded / (sum_refunded + sum_got)# ...
res_metrics['Refund_rate']

0.12453373534944157

Rebill rate:
Rebill rate measures the number of times the average user will pay us for his recurring subscription. To calculate this metric over a 6-month horizon let’s assume that it will decrease each month in 0.7 times starting from the 2nd recurrent payment.

In [95]:
num_first_month, num_second_month = 0, 0
for cust in customer_dict:
  if len(customer_dict[cust]) > 1 and customer_dict[cust].count(29.99) >= 1:
      num_first_month += 1
    # if len(customer_dict[cust]) > 2:
    #   if customer_dict[cust][2] == 29.99:
    #     num_second_month += 1
  #   num_second_month += 1
  # if 29.99 in customer_dict[cust]:
  #   num_first_month += 1
  if len(customer_dict[cust]) > 2 and customer_dict[cust].count(29.99) >= 2:
    num_second_month += 1

print(num_first_month, num_second_month)


monthly_payers = [round(num_second_month * 0.7)]

for _ in range(4):
  monthly_payers.append(round(monthly_payers[-1] * 0.7))

print(monthly_payers)

print(num_first_month, num_second_month)

print(num_second_month / num_first_month)

res_metrics['Rebill_rate'] = (sum(monthly_payers) + num_second_month + num_first_month) / num_first_month# ...
res_metrics['Rebill_rate']

5098 2472
[1730, 1211, 848, 594, 416]
5098 2472
0.48489603766182815


2.4262455865045114

And add the commission (10%)

In [97]:
res_metrics['Commisions'] = 0.1

In [98]:
res_metrics

{'CR_from_trial': 0.6053309552120492,
 'Refund_rate': 0.12453373534944157,
 'Rebill_rate': 2.4262455865045114,
 'Commisions': 0.1}

## Calculating LTV
(0.99 + 29.99 * AA % * (1- BB %) * С.СС) * (1 - ZZ %) with AA % being the conversion rate, BB % - the refund rate, C.CC - the rebill rate and ZZ % - the commissions.

In [99]:
LTV = (0.99 + 29.99 * res_metrics["CR_from_trial"] * (1 - res_metrics["Refund_rate"]) * res_metrics["Rebill_rate"]) * (1 - res_metrics['Commisions'])# ...

In [100]:
LTV

35.595519234072064